In [ ]:
import sys
sys.path.append("..")
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
import nidaqmx
import numpy as np
# from IPython.display import display, clear_output
# from skimage import data
# from skimage.feature import match_template

from LowLevelModules.NIdaq import *
from LowLevelModules.NIdaqAPD import *
from LowLevelModules.GeneralFunctions import *

#from collections import deque

class FSM:
    def __init__(self):
        
        self.go_to_position(0, 0)
        self.curr_pos = {'x': 0, 'y': 0}
        self.conversion = {'x': 13.90, 'y': 9.69} # um/volt ???

    def micron_to_volts(self, pos_in_micron, axis):
        return pos_in_micron / self.conversion[axis]

    def volts_to_micron(self, pos_in_volts, axis):
        return pos_in_volts * self.conversion[axis]

    def read_position(self, unit='volts'):
        """ Voltage reading of the 2 AI from the DAQ
            Returns in Volts unless asked otherwise """
        with nidaqmx.Task() as fsm_task:
            fsm_task.ai_channels.add_ai_voltage_chan('/Dev1/ai1', 'FSM x axis')
            fsm_task.ai_channels.add_ai_voltage_chan('/Dev1/ai0', 'FSM y axis')
            curr_x, curr_y = fsm_task.read()
        if unit != 'volts':
            curr_x = self.volts_to_micron(curr_x, 'x')
            curr_y = self.volts_to_micron(curr_y, 'y')
        return curr_x, curr_y

    def calc_trajectory_in_V(self, final_pos, n_pts, axis):
        """  Returns an array in volts of a sine-like profile 
             for sweeping the values from one position to the next """
        init_pos = self.curr_pos[axis]
        delta_pos = self.micron_to_volts(new_y, axis) - init_pos
        x_sine = np.linspace(-np.pi/2, np.pi/2, n_pts)
        sweep_array = (np.sin(x_sine) + 1)/2*(final_pos-init_pos) + init_pos
        return sweep_array

    def go_to_position(self, x, y):
        """ Smooth sinusoidal move from current position to (x,y) in um """
        #curr_x, curr_y = read_position('microns')
        nx = int(np.ceil((abs(x-self.curr_pos['x'])*500+10)/2))
        ny = int(np.ceil((abs(y-self.curr_pos['y'])*500+10)/2))    
        n_max = max(nx,ny)

        # 2 arrays need to have the same size
        xVsweep_array = calc_trajectory_in_V(x, n_max, 'x')
        yVsweep_array = calc_trajectory_in_V(y, n_max, 'y')

        with nidaqmx.Task() as task:
            task.ao_channels.add_ao_voltage_chan( '/Dev1/ao0', 'FSM y axis', min_val=-9, max_val=9)
            task.ao_channels.add_ao_voltage_chan('/Dev1/ao1', 'FSM x axis', min_val=-9, max_val=9)
            task.timing.cfg_samp_clk_timing(rate= 50000, 
                                            active_edge=nidaqmx.constants.Edge.RISING ,
                                            sample_mode= nidaqmx.constants.AcquisitionType.FINITE, 
                                            samps_per_chan=len(xVsweep_array))
            task.write(np.array([yVsweep_array, xVsweep_array]), auto_start=True, timeout=5)
            task.wait_until_done(timeout=5)

        # set the new current position
        self.curr_pos['x'], self.curr_pos['y'] = self.read_position()
        

def scan_1D(init, final, mesh_pts, scan_rate, axis, counter_terminal= "/Dev1/PFI0"):
    """
    Scan 1D
    
    Parameters:
    init [um]
    final [um]
    no. points
    rate (Hz)
    axis
    APD terminal
    
    Returns:
    (X [um],APD counts [kCt/s]) list
    """
    counterPort = "/Dev1/ctr1"

    # don't go singificantly above 200 Hz, else the FSM may not be able to catch up
    # such that the AO step by 0.3mV = 20V/2^16 (16 bit output -10 to +10 V)
    mult_fac = 5
    
    ao_scan_rate = scan_rate*mult_fac
    ao_pts = mesh_pts*mult_fac

    # x trajectory 
    axis_scan = np.linspace(init, final, mesh_pts)
    
    # voltage scan is denser than the mesh    
    v_ao_scan = micron_to_volts(np.linspace(init, final, ao_pts), axis)
    
    # clock source
    counter_term_clk = '/Dev1/ctr0'
    trig_src =  '/Dev1/PFI12'       

    # setup counter
    countAPD = CI(counter_terminal, counterPort)
    countAPD.config_read_samples(trig_src,mesh_pts+1,scan_rate)        

    # set up FSM AO AI
    FSMXtaskb=bAO('/Dev1/ao1')
    FSMXtask.config_write(v_ao_scan, ao_scan_rate, trig_src)

    FSMreadXtask = AI('/Dev1/ai1')
    FSMreadXtask.config_read(ao_pts, ao_scan_rate, trig_src)

    # PFI12 start
    # CREATE EXT CLOCK TO GATE THE READING OF PULSES COMING FROM THE APD
    ext_clock_task = CO(counter_term_clk, scan_rate)

    # wait until done and read data
    FSMXtask.wait_until_done()
    FSMXtask.close()

    aiV=FSMreadXtask.read(number_of_samples_per_channel=ao_pts)
    thisX = volts_to_micron(np.asarray(aiV), axis)
    downSampleX = thisX[::mult_fac]
    FSMreadXtask.close()

    rawAPD=countAPD.read(number_of_samples_per_channel=mesh_pts+1)
    correctAPD = np.diff(rawAPD)*scan_rate/1000 # kHz

    countAPD.close()
    ext_clock_task.close()
    return downSampleX,correctAPD     

In [ ]:
def scan_2D(curr_x,curr_y,x_size=20,y_size=20,mesh_x=20,mesh_y=20,scan_rate=100):
    """  
    Line by line live plot
    
    Parameters: 
    Returns:
    xx
    yy
    FSM2D (array): Scan image in kHz
    Data format
    FSM[row,col] corresponds to coordinate (X=xx[col],Y=yy[row])    
    """        
    # rectangular scan

    # scan left to right, top to bottom
    startX = curr_x - x_size/2
    endX = curr_x + x_size/2

    startY = curr_y + y_size/2
    endY = curr_y - y_size/2
    
    x_scan = np.linspace(startX,endX, mesh_x)
    y_scan = np.linspace(startY,endY, mesh_y)

    FSM2D = np.zeros((mesh_x,mesh_y))
    
    # similar to LabView control
    # software loop in y
    # hardware loop in X

#     fig = plt.figure()
#     ax = fig.add_subplot(1, 1, 1) 
    lp = LivePlot(1, 1, 5, 3, 'o', 'Coordinate (um)', 'APD counts (kHz)')
    lp2d = LivePlot2DV2( x_scan, y_scan, FSM2D, x_ext=6, y_ext=6)
    
    # Plot 1D scan and 2D plot in real time
    
    for idx,y0 in enumerate(y_scan):
        try:
            go_to_position(startX,y0)

            # setup counter
            xx,correctAPD = scan_1D(startX,endX,mesh_x,scan_rate,'x')
#             FSM2D = np.vstack([FSM2D,correctAPD])
            FSM2D[idx,:]=correctAPD
#             print(idx,y0)
            lp.plot_live(xx, correctAPD)
            lp2d.plot_live(FSM2D)
    #         t = deque(maxlen=max_data_before_refresh)
    #         cts = deque(maxlen=max_data_before_refresh)

            # append new data and plot
    #         t.append(i*delta_t)
    #         cts.append(current_cts)
            #if not i%int(frequency/2):
    #         lp.plot_live(x_scan, thisAPD)

    #         ax.set_xlim(0, i)

#             ax.cla()
#             ax.plot(x_scan, correctAPD)
#             display(fig)

#             clear_output(wait = True)
#             plt.pause(0.5)   

#             countAPD.close()

#             ext_clock_task.close()

        except KeyboardInterrupt:
            # press the stop button to trigger this
#             FSMXtask.close()
#             FSMreadXtask.close()
#             countAPD.close()
#             ext_clock_task.close()
            break
    # back to initial location
    go_to_position(curr_x,curr_y)        
    return x_scan,y_scan,FSM2D

# def scan_2D(curr_x,curr_y,x_size=40,y_size=40,mesh_x=40,mesh_y=40,scan_rate=100):
#     """  
#     Line by line live plot
    
#     Parameters: 
#     Returns: 
#     FSM2D (array): Scan image in kHz
#     """        
# #     go_to_position(0,0)
#     # rectangular scan
#     counter_terminal  = "/Dev1/PFI0"
#     counter_number = "/Dev1/ctr1"
# #     curr_x,curr_y = read_position()

#     # don't go singificantly above 200 Hz, else the FSM may not be able to catch up
#     ao_scan_rate = 200 # such that the AO step by 0.3mV = 20V/2^16 (16 bit output -10 to +10 V)

#     # scan left to right, top to bottom
#     startX = curr_x - x_size/2
#     endX = curr_x + x_size/2

#     startY = curr_y + y_size/2
#     endY = curr_y - y_size/2
#     scan_pts = 100

#     # x trajectory 
#     x_scan = np.linspace(startX,endX, mesh_x)
#     # voltage scan is denser than the actual X mesh
#     v_ao_scan = micron_to_volts(np.linspace(startX,endX, scan_pts),'x')

#     y_scan = np.linspace(startY,endY, mesh_y)

#     counter_term_clk = '/Dev1/ctr0'
#     trig_src =  '/Dev1/PFI12'   
#     counterPort = '/Dev1/ctr1'
#     counter_term = '/Dev1/PFI0' 
#     FSM2D = np.empty((0,mesh_x))
    
#     # lp = LivePlot(1, 1, 8, 5, 'o', 'Time (s)', 'APD counts (kHz)')
#     # similar to LabView control
#     # software loop in y
#     # hardware loop in X


#     fig = plt.figure()
#     ax = fig.add_subplot(1, 1, 1) 
#     # Plot 1D scan and 2D plot in real time
#     for y0 in y_scan:    
#         try:
#             go_to_position(startX,y0)
#             print(y0)
#             # setup counter
#             countAPD = CI(counter_term, counterPort)
#             countAPD.config_read_samples(trig_src,mesh_x+1,scan_rate)        

#             # set up FSM AO AI
#             FSMXtask=AO('/Dev1/ao1')
#             FSMXtask.config_write(v_ao_scan,ao_scan_rate,trig_src)

#             FSMreadXtask = AI('/Dev1/ai1')
#             FSMreadXtask.config_read(scan_pts,ao_scan_rate,trig_src)

#             # PFI12 start
#             # CREATE EXT CLOCK TO GATE THE READING OF PULSES COMING FROM THE APD
#             ext_clock_task = CO(counter_term_clk, scan_rate)

#             # wait until done and read data
#             FSMXtask.wait_until_done()
#             FSMXtask.close()

#             thisX=FSMreadXtask.read(number_of_samples_per_channel=scan_pts)
#             FSMreadXtask.close()

#             thisAPD=countAPD.read(number_of_samples_per_channel=mesh_x+1)
#             correctAPD = np.diff(thisAPD)

#             FSM2D = np.vstack([FSM2D,correctAPD])

#     #         t = deque(maxlen=max_data_before_refresh)
#     #         cts = deque(maxlen=max_data_before_refresh)

#             # append new data and plot
#     #         t.append(i*delta_t)
#     #         cts.append(current_cts)
#             #if not i%int(frequency/2):
#     #         lp.plot_live(x_scan, thisAPD)

#     #         ax.set_xlim(0, i)

#             ax.cla()
#             ax.plot(x_scan, correctAPD)
#             display(fig)

#             clear_output(wait = True)
#             plt.pause(0.5)   

#             countAPD.close()

#             ext_clock_task.close()

#         except KeyboardInterrupt:
#             # press the stop button to trigger this
#             FSMXtask.close()
#             FSMreadXtask.close()
#             countAPD.close()
#             ext_clock_task.close()
#             break
#     go_to_position(curr_x,curr_y)        
#     return FSM2D